In [23]:
from __future__ import print_function

import logging
from optparse import OptionParser
import sys
from time import time

import requests, json
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

Retrieve Remote Abstracts using EuropePMC Rest Services 
Disease Name - Diabetes
Result Type - Core (to get metadata information containing abstracts,title,etc)
Result Format - JSON



In [25]:
abstract_list = list()
for x in range(1, 10):
   
    europePMC_url = "http://www.ebi.ac.uk/europepmc/webservices/rest/search?query=diabetes&format=json&resulttype=core&pageSize=1000&page=%s"%x
    print(europePMC_url)
    r = requests.get(europePMC_url)
    data = json.loads(r.content)
    for result in data['resultList']['result']:
    
        abstract = result.get('abstractText',None)
    
        if abstract is not None:
            abstract_list.append(abstract)
      


http://www.ebi.ac.uk/europepmc/webservices/rest/search?query=diabetes&format=json&resulttype=core&pageSize=1000&page=1
http://www.ebi.ac.uk/europepmc/webservices/rest/search?query=diabetes&format=json&resulttype=core&pageSize=1000&page=2
http://www.ebi.ac.uk/europepmc/webservices/rest/search?query=diabetes&format=json&resulttype=core&pageSize=1000&page=3
http://www.ebi.ac.uk/europepmc/webservices/rest/search?query=diabetes&format=json&resulttype=core&pageSize=1000&page=4
http://www.ebi.ac.uk/europepmc/webservices/rest/search?query=diabetes&format=json&resulttype=core&pageSize=1000&page=5
http://www.ebi.ac.uk/europepmc/webservices/rest/search?query=diabetes&format=json&resulttype=core&pageSize=1000&page=6
http://www.ebi.ac.uk/europepmc/webservices/rest/search?query=diabetes&format=json&resulttype=core&pageSize=1000&page=7
http://www.ebi.ac.uk/europepmc/webservices/rest/search?query=diabetes&format=json&resulttype=core&pageSize=1000&page=8
http://www.ebi.ac.uk/europepmc/webservices/rest/

Print Total number of abstracts retrieved

In [28]:
print("%d abstracts" % len(abstract_list))

7463 abstracts


Preprocess/Clean the abstract data using spaCy NLP
1) Tokenize the abstract data
2) Lemmatize the tokens
3) Clean the tokens by applying stoplists

In [29]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from spacy import en
import string
from spacy.en import English

In [32]:
parser = English()

# A custom stoplist
STOPLIST = set()
STOPLIST.update(en.STOPWORDS)
STOPLIST.update(["n't", "'s", "'m", "ca","p"])
STOPLIST.update(list(ENGLISH_STOP_WORDS))
# List of symbols we don't care about
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...",  "'ve"] 
def tokenize_text(sample):

        # get the tokens using spaCy
        tokens = parser(sample)

        # lemmatize
        lemmas = []
        for tok in tokens:
            lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
            tokens = lemmas

        # stoplist the tokens
        tokens = [tok for tok in tokens if tok not in STOPLIST]

        # stoplist symbols
        tokens = [tok for tok in tokens if tok not in SYMBOLS]

        # remove large strings of whitespace
        while "" in tokens:
            tokens.remove("")
        while " " in tokens:
            tokens.remove(" ")
        while "\n" in tokens:
            tokens.remove("\n")
        while "\n\n" in tokens:
            tokens.remove("\n\n")
        #print tokens
        return tokens

Convert above training corpus to sparse vectors using a TF-IDF sparse vectorizer and extract relevant features
1) No. of features = 100000
2) Remove Ascii Characters from Training Corpus 

In [33]:
t0 = time()
vectorizer = TfidfVectorizer(max_df=0.5, max_features=100000,
                                 min_df=2,tokenizer=tokenize_text, stop_words='english',
                                 use_idf=True,strip_accents =  'ascii')
X = vectorizer.fit_transform(abstract_list)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)

done in 141.067945s
n_samples: 7463, n_features: 19382


Do the Actual Clustering using Scikit-Learn, KMeans Algo

In [34]:
km = KMeans(n_clusters=5, init='k-means++', max_iter=100, n_init=1,
                verbose=False)
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

Clustering sparse data with KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=5, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=False)
done in 1.185s



Print the top terms per cluster

In [35]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(5):
    print("Cluster %d:" % i, end='')
    print("Label %d:" %km.labels_[i],end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0:Label 4: obesity weight surgery loss obese metabolic bmi fat body bariatric
Cluster 1:Label 1: cell mouse diabetic insulin rat expression induce protein islet glucose
Cluster 2:Label 4: care self health management intervention use cost control adherence 2
Cluster 3:Label 4: risk 95 ci high woman age associate year 2 diabetic
Cluster 4:Label 2: insulin glucose hba1c treatment 2 control group hypoglycemia level therapy


Clustering using Mini Batch KMeans

In [36]:
km = MiniBatchKMeans(n_clusters=5, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000, verbose=False)
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

Clustering sparse data with MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
        init_size=1000, max_iter=100, max_no_improvement=10, n_clusters=5,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=False)
done in 0.226s



In [37]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(5):
    print("Cluster %d:" % i, end='')
    print("Label %d:" %km.labels_[i],end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0:Label 2: cell insulin mouse glucose induce expression diabetic effect rat protein
Cluster 1:Label 0: diabetic dr retinopathy foot complication disease retinal eye neuropathy risk
Cluster 2:Label 0: care gdm hba1c insulin control intervention health self management 2
Cluster 3:Label 0: group level glucose 2 control insulin serum t2dm child 1
Cluster 4:Label 2: risk 95 ci year high prevalence associate age 2 mortality


Bi-gram clustering???

In [38]:
t0 = time()
vectorizer = TfidfVectorizer(max_df=0.5, max_features=100000,
                                 min_df=2,tokenizer=tokenize_text, stop_words='english',
                                 use_idf=True,strip_accents =  'ascii',ngram_range=(1,2))
X = vectorizer.fit_transform(abstract_list)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
km = KMeans(n_clusters=5, init='k-means++', max_iter=100, n_init=1,
                verbose=False)
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(5):
    print("Cluster %d:" % i, end='')
    print("Label %d:" %km.labels_[i],end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

done in 148.384284s
n_samples: 7463, n_features: 100000
Clustering sparse data with KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=5, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=False)
done in 3.501s

Cluster 0:Label 3: cell islet beta cell beta t cell t t1d mouse insulin pancreatic
Cluster 1:Label 4: ci 95 95 ci risk gdm woman pregnancy gestational year hr
Cluster 2:Label 2: risk diabetic disease level 2 high group age type 2 associate
Cluster 3:Label 4: care insulin hba1c self control health management 2 use intervention
Cluster 4:Label 3: insulin mouse rat diabetic glucose induce expression cell effect increase
